In [1]:
import os   #used for directory work
import matplotlib.pyplot as plt #used for creating plots
import pandas as pd
import numpy as np
%matplotlib inline

In [3]:
def text_finder(path):
    txt_files=[]
    all_files = os.listdir(path) #tells what directory the files are in
    for i in all_files: #reading only the .txt files 
        if i[-4:] == ".txt":
            txt_files.append(i)
    txt_files.sort() #alphabetizes them
    return txt_files

def file_to_df(path):
    df= pd.read_csv(path,na_values = ['NAN', "00nan", "NaN"], sep=" ")
    
    return df

In [61]:
path = "/home/joey/Desktop/10X10_Truss_SERDP_Burns/JS_Cleaned_Data/Full-Timestamp/Burn-02/sonics/"
txt_files = text_finder(path)[:16]


In [74]:
def find_continuous( col,burn, index = 0 ):
    path = "/home/joey/Desktop/10X10_Truss_SERDP_Burns/JS_Cleaned_Data/Full-Timestamp/Burn-"+burn+"/sonics/"
    txt_files = text_finder(path)[:16]

    check, index = True, 0
    while check:
        for f in range(16):
            df=file_to_df(path+txt_files[f])

            lst =pd.Series(df[col][index:index+6000]).isnull()
            if len(lst.unique())==1:
                if f == 15:
                    #print("Passed", index)
                    check = False
                    break

                else:
                    continue
            i_lst = []    
            if len(lst.unique())!=1:
                for i in range(index,index+6000):
                    if lst[i]==True:
                        index= i+1
                        i_lst.append(i)
                index = max(i_lst)+1
                break
    return index

def index_getter(burn): 
    check = True
    while check:
        U_start = find_continuous("U",burn)
        V_start = find_continuous("V",burn, U_start)
        if V_start > U_start:
            continue
        W_start = find_continuous("W", burn, V_start)
        if W_start > V_start:
            continue
        T_start = find_continuous("T",burn, W_start)
        if T_start > W_start:
            continue
        check = False
    return T_start

In [78]:
#indx_01 = index_getter("01")
indx_02 =index_getter("02")
print(indx_02)

3795


In [139]:
burn = "02"
indx_02 =index_getter(burn) 
path = "/home/joey/Desktop/10X10_Truss_SERDP_Burns/JS_Cleaned_Data/Full-Timestamp/Burn-"+burn+"/sonics/"
txt_files = text_finder(path)[:16]

df = file_to_df(path+txt_files[0])
Hr_lst, Min_lst, Sec_lst = [], [], []
for i in range(indx_02,indx_02+6000):
    Hr_lst.append(int(df["TIMESTAMP"][i][11:13]))
    Min_lst.append(int(df["TIMESTAMP"][i][14:16]))
    Sec_lst.append(float(df["TIMESTAMP"][i][17:21]))
    
d ={"Hr": Hr_lst, "Min": Min_lst, "Sec": Sec_lst}
df_U, df_V = pd.DataFrame(d), pd.DataFrame(d)
df_W, df_T = pd.DataFrame(d), pd.DataFrame(d)

for f in range(16):
    df_U[txt_files[f][:2]+"_U"] =list(file_to_df(path+txt_files[f])["U"][indx_02:indx_02+6000])
    df_V[txt_files[f][:2]+"_V"] =list(file_to_df(path+txt_files[f])["V"][indx_02:indx_02+6000])
    df_W[txt_files[f][:2]+"_W"] =list(file_to_df(path+txt_files[f])["W"][indx_02:indx_02+6000])
    df_T[txt_files[f][:2]+"_T"] =list(file_to_df(path+txt_files[f])["T"][indx_02:indx_02+6000])


In [147]:
df_U.to_csv("10min-Cont-Sample-Burn-"+burn+"-U", sep=' ',index=False)
df_V.to_csv("10min-Cont-Sample-Burn-"+burn+"-V", sep=' ',index=False)
df_W.to_csv("10min-Cont-Sample-Burn-"+burn+"-W", sep=' ',index=False)
df_T.to_csv("10min-Cont-Sample-Burn-"+burn+"-T", sep=' ',index=False)

In [126]:

df_test = pd.DataFrame(d)
df_test
#df_test = pd.DataFrame(data = np.array([Hr_lst, Min_lst, Sec_lst]), columns = ["Hr", "Min", "Sec"])
#df_T["Hr"] = Hr_lst
#np.array([Hr_lst, Min_lst, Sec_lst])

Hr  Min   Sec
0     11   10  46.4
1     11   10  46.5
2     11   10  46.6
3     11   10  46.7
4     11   10  46.8
...   ..  ...   ...
5995  11   20  45.9
5996  11   20  46.0
5997  11   20  46.1
5998  11   20  46.2
5999  11   20  46.3

[6000 rows x 3 columns]

In [117]:
df_test

NameError: name 'df_test' is not defined